# TASK 1

In [1]:
#IMPORTING ALL THE LIBRARIES
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, BooleanType
from pyspark.sql.types import StructType, StructField, TimestampType, DateType, StringType
from pyspark.sql.functions import col, trim
from pyspark.sql.functions import unix_timestamp, current_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import count, avg, first, year
import seaborn as sb
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
import matplotlib.pyplot as plt
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import roc_curve
import pyspark.sql.functions as F
import pyspark.sql.types as T
from matplotlib import pyplot as plt
import pandas as pd
from pyspark.sql.functions import col, sum, when
import torch
import numpy as np
import pandas as pd
import itertools
from torch.utils.data import DataLoader
from torch import nn, optim

In [31]:
#DOWNLOADING THE DATASET FROM GITHUB REPOSITORY
!wget "https://github.com/jjanbol/Test-Toolchains/raw/refs/heads/main/Archive.zip"

/bin/bash: wget: command not found


In [32]:
#UNZIPPING FOR DATA INGESTION
!unzip -o Archive.zip

unzip:  cannot find or open Archive.zip, Archive.zip.zip or Archive.zip.ZIP.


In [2]:
#CHECKING THAT FILES WERE UNZIPPED CORRECTLY
!ls

Archive.zip                      female_players_21.csv
Project_cloud.ipynb              female_players_22.csv
Project_option_1_rubric.docx.pdf players_15.csv
__MACOSX                         players_16.csv
archive                          players_17.csv
best_model                       players_18.csv
best_model2                      players_19.csv
code files                       players_20.csv
current_best_model               players_21.csv
female_players_16.csv            players_22.csv
female_players_17.csv            postgresql-42.7.4.jar
female_players_18.csv            project v1.0.ipynb
female_players_19.csv            project.ipynb
female_players_20.csv


In [3]:
# INITIALIZING SPARK SESSION 
jdbc_jar_path = "postgresql-42.7.4.jar"

appName = "Project"
master = "local"

conf = SparkConf() \
    .set('spark.driver.host', '127.0.0.1') \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.jars", jdbc_jar_path)

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("FATAL")


24/11/14 01:14:07 WARN Utils: Your hostname, Janbols-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.0.0.64 instead (on interface en0)
24/11/14 01:14:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/11/14 01:14:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/opt/anaconda3/envs/VENV_Apache/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
#DEFINING FUNCTION TO CREATE DATAFRAMES

schema = StructType([
    StructField("sofifa_id", IntegerType(), True),  
    StructField("player_url", StringType(), True),  
    StructField("short_name", StringType(), True),  
    StructField("long_name", StringType(), True),  
    StructField("player_positions", StringType(), True), 
    StructField("overall", IntegerType(), True),  
    StructField("potential", IntegerType(), True),
    StructField("value_eur", IntegerType(), True),  
    StructField("wage_eur", IntegerType(), True),  
    StructField("age", IntegerType(), True),  
    StructField("dob", DateType(), True),  
    StructField("height_cm", IntegerType(), True),  
    StructField("weight_kg", IntegerType(), True),  
    StructField("club_team_id", IntegerType(), True),
    StructField("club_name", StringType(), True),  
    StructField("league_name", StringType(), True),
    StructField("league_level", IntegerType(), True), 
    StructField("club_position", StringType(), True),  
    StructField("club_jersey_number", IntegerType(), True),  
    StructField("club_loaned_from", StringType(), True),  
    StructField("club_joined", DateType(), True),  
    StructField("club_contract_valid_until", DateType(), True),  
    StructField("nationality_id", IntegerType(), True),  
    StructField("nationality_name", StringType(), True), 
    StructField("nation_team_id", IntegerType(), True),  
    StructField("nation_position", StringType(), True),  
    StructField("nation_jersey_number", IntegerType(), True),  
    StructField("preferred_foot", StringType(), True),  
    StructField("weak_foot", IntegerType(), True),  
    StructField("skill_moves", IntegerType(), True), 
    StructField("international_reputation", IntegerType(), True),  
    StructField("work_rate", StringType(), True),  
    StructField("body_type", StringType(), True),  
    StructField("real_face", BooleanType(), True),  
    StructField("release_clause_eur", IntegerType(), True),  
    StructField("player_tags", StringType(), True),  
    StructField("player_traits", StringType(), True),  
    StructField("pace", IntegerType(), True),  
    StructField("shooting", IntegerType(), True),
    StructField("passing", IntegerType(), True), 
    StructField("dribbling", IntegerType(), True),
    StructField("defending", IntegerType(), True),  
    StructField("physic", IntegerType(), True),  
    StructField("attacking_crossing", IntegerType(), True),  
    StructField("attacking_finishing", IntegerType(), True),  
    StructField("attacking_heading_accuracy", IntegerType(), True),  
    StructField("attacking_short_passing", IntegerType(), True),  
    StructField("attacking_volleys", IntegerType(), True),  
    StructField("skill_dribbling", IntegerType(), True),  
    StructField("skill_curve", IntegerType(), True),  
    StructField("skill_fk_accuracy", IntegerType(), True),  
    StructField("skill_long_passing", IntegerType(), True),  
    StructField("skill_ball_control", IntegerType(), True),  
    StructField("movement_acceleration", IntegerType(), True),  
    StructField("movement_sprint_speed", IntegerType(), True),  
    StructField("movement_agility", IntegerType(), True),  
    StructField("movement_reactions", IntegerType(), True),  
    StructField("movement_balance", IntegerType(), True),  
    StructField("power_shot_power", IntegerType(), True),  
    StructField("power_jumping", IntegerType(), True),  
    StructField("power_stamina", IntegerType(), True),  
    StructField("power_strength", IntegerType(), True), 
    StructField("power_long_shots", IntegerType(), True),  
    StructField("mentality_aggression", IntegerType(), True),
    StructField("mentality_interceptions", IntegerType(), True),
    StructField("mentality_positioning", IntegerType(), True),  
    StructField("mentality_vision", IntegerType(), True),  
    StructField("mentality_penalties", IntegerType(), True),
    StructField("mentality_composure", IntegerType(), True), 
    StructField("defending_marking_awareness", IntegerType(), True),  
    StructField("defending_standing_tackle", IntegerType(), True),  
    StructField("defending_sliding_tackle", IntegerType(), True),  
    StructField("goalkeeping_diving", IntegerType(), True),  
    StructField("goalkeeping_handling", IntegerType(), True),
    StructField("goalkeeping_kicking", IntegerType(), True),  
    StructField("goalkeeping_positioning", IntegerType(), True),  
    StructField("goalkeeping_reflexes", IntegerType(), True),  
    StructField("goalkeeping_speed", IntegerType(), True),  
    StructField("ls", IntegerType(), True),  
    StructField("st", IntegerType(), True),  
    StructField("rs", IntegerType(), True),  
    StructField("lw", IntegerType(), True),  
    StructField("lf", IntegerType(), True),  
    StructField("cf", IntegerType(), True),  
    StructField("rf", IntegerType(), True),  
    StructField("rw", IntegerType(), True),  
    StructField("lam", IntegerType(), True), 
    StructField("cam", IntegerType(), True), 
    StructField("ram", IntegerType(), True), 
    StructField("lm", IntegerType(), True),  
    StructField("lcm", IntegerType(), True), 
    StructField("cm", IntegerType(), True),  
    StructField("rcm", IntegerType(), True), 
    StructField("rm", IntegerType(), True),  
    StructField("lwb", IntegerType(), True), 
    StructField("ldm", IntegerType(), True), 
    StructField("cdm", IntegerType(), True), 
    StructField("rdm", IntegerType(), True), 
    StructField("rwb", IntegerType(), True), 
    StructField("lb", IntegerType(), True),  
    StructField("lcb", IntegerType(), True),  
    StructField("cb", IntegerType(), True),  
    StructField("rcb", IntegerType(), True), 
    StructField("rb", IntegerType(), True),  
    StructField("gk", IntegerType(), True),  
    StructField("player_face_url", StringType(), True), 
    StructField("club_logo_url", StringType(), True),  
    StructField("club_flag_url", StringType(), True),  
    StructField("nation_logo_url", StringType(), True),
    StructField("nation_flag_url", StringType(), True)
])


files = ["female_players_16.csv", "female_players_17.csv", "female_players_18.csv", "female_players_19.csv", 
         "female_players_20.csv", "female_players_21.csv", "female_players_22.csv", 
         "players_15.csv", "players_16.csv", "players_17.csv", "players_18.csv", "players_19.csv", 
         "players_20.csv", "players_21.csv", "players_22.csv"]

def create_dataframe(csv):
    directory = csv
    years = ["15", "16", "17", "18", "19", "20", "21", "22"]
    for year in years:
        if (year in csv) and ("female" in csv):
            df = spark.read.csv(directory,header=True, schema= schema)
            df = df.withColumn('year', lit(int("20"+year)))
            df = df.withColumn('gender', lit("female"))
            return df
        elif (year in csv) and ("female" not in csv):
            df = spark.read.csv(directory,header=True, schema= schema)
            df = df.withColumn('year', lit(int("20"+year)))
            df = df.withColumn('gender', lit("male"))
            return df
                               

#CREATING DATAFRAMES WITH YEAR AND GENDER COLUMNS
female_players_16 = create_dataframe("female_players_16.csv")
female_players_17 = create_dataframe("female_players_17.csv")
female_players_18 = create_dataframe("female_players_18.csv")
female_players_19 = create_dataframe("female_players_19.csv")
female_players_20 = create_dataframe("female_players_20.csv")
female_players_21 = create_dataframe("female_players_21.csv")
female_players_22 = create_dataframe("female_players_22.csv")
players_15 = create_dataframe("players_15.csv")
players_16 = create_dataframe("players_16.csv")
players_17 = create_dataframe("players_17.csv")
players_18 = create_dataframe("players_18.csv")
players_19 = create_dataframe("players_19.csv")
players_20 = create_dataframe("players_20.csv")
players_21 = create_dataframe("players_21.csv")
players_22 = create_dataframe("players_22.csv")


dataframes = [female_players_16, female_players_17,female_players_18 ,female_players_19,
              female_players_20 ,female_players_21,female_players_22,
              players_15, players_16, players_17, players_18, players_19, players_20,
              players_21, players_22]


In [62]:
# WRITING THE DATAFRAME TO GOOGLE CLOUD SQL
db_url = "jdbc:postgresql://34.135.160.25:5432/postgres" 

db_properties = {
    "user": "postgres",
    "password": "Tools2024",
    "driver": "org.postgresql.Driver"
}
for df in dataframes:
    df.write.jdbc(url=db_url, table="fifa.players", mode="append", properties=db_properties)

In [4]:
# PULLING THE DATAFRAME FROM GOOGLE CLOUD SQL AS A TEST
db_url = "jdbc:postgresql://34.135.160.25:5432/postgres" 
db_properties = {
    "user": "postgres",
    "password": "Tools2024",
    "driver": "org.postgresql.Driver"
}
df = spark.read.jdbc(url=db_url, table="fifa.players", properties=db_properties)
# CACHING DATAFRAME TO MEMORY, THIS WILL IMPROVE PERFORMANCE
df.cache()  

DataFrame[unique_id: int, sofifa_id: int, player_url: string, short_name: string, long_name: string, player_positions: string, overall: int, potential: int, value_eur: int, wage_eur: int, age: int, dob: date, height_cm: int, weight_kg: int, club_team_id: int, club_name: string, league_name: string, league_level: int, club_position: string, club_jersey_number: int, club_loaned_from: string, club_joined: date, club_contract_valid_until: date, nationality_id: int, nationality_name: string, nation_team_id: int, nation_position: string, nation_jersey_number: int, preferred_foot: string, weak_foot: int, skill_moves: int, international_reputation: int, work_rate: string, body_type: string, real_face: boolean, release_clause_eur: int, player_tags: string, player_traits: string, pace: int, shooting: int, passing: int, dribbling: int, defending: int, physic: int, attacking_crossing: int, attacking_finishing: int, attacking_heading_accuracy: int, attacking_short_passing: int, attacking_volleys: i

In [5]:
df.columns

['unique_id',
 'sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'club_team_id',
 'club_name',
 'league_name',
 'league_level',
 'club_position',
 'club_jersey_number',
 'club_loaned_from',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',


# TASK 2

In [6]:
# INITIALIZING SPARK SESSION 
jdbc_jar_path = "postgresql-42.7.4.jar"

appName = "Project"
master = "local"

conf = SparkConf() \
    .set('spark.driver.host', '127.0.0.1') \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.jars", jdbc_jar_path)

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession.builder.getOrCreate()

## PULLING THE DATAFRAME FROM GOOGLE CLOUDSQL

In [7]:
#pulling the data from CloudSQL
db_url = "jdbc:postgresql://34.135.160.25:5432/postgres"

db_properties = {
    "user": "postgres",
    "password": "Tools2024",
    "driver": "org.postgresql.Driver"
}
df = spark.read.jdbc(url=db_url, table="fifa.players", properties=db_properties)

In [8]:
#CACHING DATAFRAME TO MEMORY, THIS WILL IMPROVE PERFORMANCE
df.cache()

DataFrame[unique_id: int, sofifa_id: int, player_url: string, short_name: string, long_name: string, player_positions: string, overall: int, potential: int, value_eur: int, wage_eur: int, age: int, dob: date, height_cm: int, weight_kg: int, club_team_id: int, club_name: string, league_name: string, league_level: int, club_position: string, club_jersey_number: int, club_loaned_from: string, club_joined: date, club_contract_valid_until: date, nationality_id: int, nationality_name: string, nation_team_id: int, nation_position: string, nation_jersey_number: int, preferred_foot: string, weak_foot: int, skill_moves: int, international_reputation: int, work_rate: string, body_type: string, real_face: boolean, release_clause_eur: int, player_tags: string, player_traits: string, pace: int, shooting: int, passing: int, dribbling: int, defending: int, physic: int, attacking_crossing: int, attacking_finishing: int, attacking_heading_accuracy: int, attacking_short_passing: int, attacking_volleys: i

## TASK 2 Q1

In [465]:


def contract_expiry_analysis(df,year_x,y,year_z):

    df_male = df.filter(df['gender'] == 'male') #only using the male player data

    filtered_df = df_male.filter(df_male['year'] == year_x) # Filtering the dataframe to only include the year X

    filtered_yrz_df = filtered_df.filter(year(filtered_df['club_contract_valid_until']) >= year_z)
                                                                     #filter for contracts that end in or after year Z


    club_ordered_df = filtered_yrz_df.groupBy("club_name") #grouping the filtered dataframe by the club names
    player_count_df = club_ordered_df.agg(count("*").alias("player_count")) # Counting the number of players


    ordered_players_df = player_count_df.orderBy("player_count", ascending=False) # Ordering the dataframe in descending order of player count
    limited_ordered_df = ordered_players_df.limit(y) #limiting the output to Y Clubs



    limited_ordered_df.show()


In [466]:
year_x = 2022# Enter a year X (between 2015 and 2022)
y = 5# Enter a positive integer Y - number of clubs we want to analyze
year_z = 2023 # Enter a year Z, contract end year
contract_expiry_analysis(df, year_x,y,year_z)

+--------------------+------------+
|           club_name|player_count|
+--------------------+------------+
|        Leeds United|          29|
|          Sevilla FC|          29|
|        FC Barcelona|          27|
|       VfB Stuttgart|          27|
|Borussia Möncheng...|          27|
+--------------------+------------+



## TASK 2 Q2

In [467]:

  
def average_age_analysis(df,x,year_y,find_lowest=True):
    if x <= 0:  # Accounting for an invalid x input
        print("Invalid x")
        return
    
    df_male = df.filter(df['gender'] == 'male') #only using the male player data
    year_filtered_df = df_male.filter(df_male['year'] == year_y) # Filtering the dataframe to only include the year Y

    club_grouped_df = year_filtered_df.groupBy("club_name") # Grouping the data frame by the names of the clubs
    
    avg_age_df = club_grouped_df.agg(avg("age").alias("average_age")) # Aggregating the average player age, and storing it under an aliased column

  
    arranged_df = avg_age_df.orderBy("average_age", ascending=find_lowest) # Arranging the average age column depending on the input condition of ascending or descending 


    limited_df = arranged_df.limit(x) #Limiting the arranged dataframe to 'X' entries

    # Accounting for multiple occurrences of the same average age for the Xth ranking

    last_avg_age = limited_df.tail(1)[0]['average_age'] #Finding the value of the Xth ranking
    
    tied_clubs_df = arranged_df.filter(arranged_df['average_age'] == last_avg_age) # Filtering the prior dataframe to include every entry that has the same value of average age
    unordered_final_df = limited_df.union(tied_clubs_df).distinct() #union of the limited dataframe with the one consisting of the tied clubs

    ordered_final_df = unordered_final_df.orderBy("average_age", ascending=find_lowest) # accounting for the case in which the union function does not order the data as we wished


    ordered_final_df.show()
    



In [468]:
x = 5# Enter a positivr integer x - number of clubs we want to rank
year_y = 2015 # Enter a year to find the data for
find_lowest = False # 'True' if we arrange in ascending, 'False' if in descending
average_age_analysis(df,x, year_y,find_lowest)

+--------------------+------------------+
|           club_name|       average_age|
+--------------------+------------------+
|           Cruz Azul|28.071428571428573|
|        Arsenal Tula|             28.04|
|Podbeskidzie Biel...|27.962962962962962|
|       Fenerbahçe SK|             27.88|
|Leones Negros de ...| 27.79310344827586|
+--------------------+------------------+



## TASK 2 Q3

In [469]:

def popular_nationality(df):

    df_male = df.filter(df['gender'] == 'male') #only using the male player data
   
    nationality_counts = df_male.groupBy("year", "nationality_name").agg(count("*").alias("nat_count")) # grouping the data by year and nationality
                                                                                                   # and counting the number of occurrences

    
    ordered_nat_df = nationality_counts.orderBy('nat_count', ascending =False) # Ordering the nationality counts in descending order

    
    final_df = ordered_nat_df.groupBy('year').agg(first('nationality_name')) # Picking out the maximum count of nationality for each year
    final_df.show()
    





In [470]:
popular_nationality(df)

+----+-----------------------+
|year|first(nationality_name)|
+----+-----------------------+
|2015|                England|
|2016|                England|
|2017|                England|
|2018|                England|
|2019|                England|
|2020|                England|
|2021|                England|
|2022|                England|
+----+-----------------------+



# TASK 3

In [9]:
# INITIALIZING SPARK SESSION 
jdbc_jar_path = "postgresql-42.7.4.jar"

appName = "Project"
master = "local"

conf = SparkConf() \
    .set('spark.driver.host', '127.0.0.1') \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.jars", jdbc_jar_path)

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("FATAL")


In [10]:
#PULLING THE DATA FROM CLOUDSQL
db_url = "jdbc:postgresql://34.135.160.25:5432/postgres"

db_properties = {
    "user": "postgres",
    "password": "Tools2024",
    "driver": "org.postgresql.Driver"
}
df = spark.read.jdbc(url=db_url, table="fifa.players", properties=db_properties)
spark.sparkContext.setLogLevel("FATAL")


## DATA ENGINEERING PROCESS

In [11]:
#Choosing only male data
df_male = df.filter(df["gender"] == "male")
df_male.cache()

DataFrame[unique_id: int, sofifa_id: int, player_url: string, short_name: string, long_name: string, player_positions: string, overall: int, potential: int, value_eur: int, wage_eur: int, age: int, dob: date, height_cm: int, weight_kg: int, club_team_id: int, club_name: string, league_name: string, league_level: int, club_position: string, club_jersey_number: int, club_loaned_from: string, club_joined: date, club_contract_valid_until: date, nationality_id: int, nationality_name: string, nation_team_id: int, nation_position: string, nation_jersey_number: int, preferred_foot: string, weak_foot: int, skill_moves: int, international_reputation: int, work_rate: string, body_type: string, real_face: boolean, release_clause_eur: int, player_tags: string, player_traits: string, pace: int, shooting: int, passing: int, dribbling: int, defending: int, physic: int, attacking_crossing: int, attacking_finishing: int, attacking_heading_accuracy: int, attacking_short_passing: int, attacking_volleys: i

In [12]:
df_male.count()

142079

In [13]:
df_male.show(1, vertical  = True)


-RECORD 0-------------------------------------------
 unique_id                   | 2245                 
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | NULL                 
 wage_eur                    | NULL                 
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | NULL                 
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1              

In [6]:
#CHOOSING ALL THE NUMBERICAL COLUMNS TO CHECK THE CORRELATION
continuous_cols = ['potential', 'age', 'height_cm', 'weight_kg', 'weak_foot', 'skill_moves','pace',
                   'shooting','passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_heading_accuracy', 
                   'attacking_volleys', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'movement_agility', 'movement_reactions', 
                   'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'mentality_aggression', 
                   'mentality_vision', 'mentality_penalties', 'mentality_composure', 'goalkeeping_diving', 
                   'goalkeeping_speed', 'mentality_positioning', 'movement_acceleration', 'movement_sprint_speed', 'attacking_finishing', 
                   'power_long_shots', 'attacking_short_passing', 'skill_dribbling', 'skill_ball_control', 'defending_marking_awareness', 
                   'mentality_interceptions', 'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_handling', 'goalkeeping_kicking', 
                   'goalkeeping_positioning', 'goalkeeping_reflexes', 'movement_acceleration', 'movement_sprint_speed']
cor_df = df_male.select(*continuous_cols) #FILTERING ON NUMERICAL COLUMNS
correlation_matrix = cor_df.toPandas().corr() #CONVERTING THE SPARK TO PANDAS AND CREATING CORRELATION MATRIX
print(correlation_matrix*100)

                              potential         age   height_cm   weight_kg  \
potential                    100.000000  -19.694003    0.505769   -0.901486   
age                          -19.694003  100.000000    8.298078   22.715296   
height_cm                      0.505769    8.298078  100.000000   76.531476   
weight_kg                     -0.901486   22.715296   76.531476  100.000000   
weak_foot                     17.024495    7.663141  -17.153295  -13.067530   
skill_moves                   28.707977    2.408352  -42.154507  -35.787364   
pace                          26.550191  -19.990491  -41.175164  -36.890854   
shooting                      32.101446   23.195762  -18.981880   -9.346540   
passing                       45.637442   33.095066  -27.234769  -18.068139   
dribbling                     48.709587   15.952591  -39.486679  -29.868601   
defending                     15.828173   25.399656   21.891134   21.312066   
physic                        18.854280   42.556781 

In [12]:
#Highlighting the correlations that have more than 90% coeffieciency.
high_correlation = correlation_matrix[correlation_matrix > 0.9]

In [13]:
print(high_correlation)

                             potential  age  height_cm  weight_kg  weak_foot  \
potential                          1.0  NaN        NaN        NaN        NaN   
age                                NaN  1.0        NaN        NaN        NaN   
height_cm                          NaN  NaN        1.0        NaN        NaN   
weight_kg                          NaN  NaN        NaN        1.0        NaN   
weak_foot                          NaN  NaN        NaN        NaN        1.0   
skill_moves                        NaN  NaN        NaN        NaN        NaN   
pace                               NaN  NaN        NaN        NaN        NaN   
shooting                           NaN  NaN        NaN        NaN        NaN   
passing                            NaN  NaN        NaN        NaN        NaN   
dribbling                          NaN  NaN        NaN        NaN        NaN   
defending                          NaN  NaN        NaN        NaN        NaN   
physic                             NaN  

## Noting which columns have high correlation.

pace -> movement_acceleration, movement_sprint_speed

shooting -> attacking_finishing, power_long_shots

passing -> attacking_short_passing

dribbling -> skill_dribbling, skill_ball_control

defending -> defending_marking_awareness, mentality_interceptions, defending_standing_tackle, defending_sliding_tackle

goalkeeping_diving -> goalkeeping_handling, goalkeeping_kicking, goalkeeping_positioning, goalkeeping_reflexes

goalkeeping_speed -> movement_acceleration, movement_sprint_speed

In [14]:
#Dr.Guannan, Lecture 9 and 10 notes.

#COLUMNS THAT WE WILL BE KEEPING. THESE ARE IMPORTANT IN PREDICTING THE OVERALL RATING OF A PLAYER
continuous_cols = ['age', 'height_cm', 'weight_kg', 'weak_foot', 'skill_moves','pace',
                   'shooting','passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_heading_accuracy', 
                   'attacking_volleys', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'movement_agility', 'movement_reactions', 
                   'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'mentality_aggression', 
                   'mentality_vision', 'mentality_penalties', 'mentality_composure', 'goalkeeping_diving', 
                   'goalkeeping_speed', 'mentality_positioning']

#includes columns that have too many Nulls and high correlation coefficient and unique identifiers and URLS. 
columns_to_drop = ['unique_id', 'sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions', 'value_eur', 'wage_eur', 'dob', 
                   'club_team_id', 'club_name', 'league_name', 'league_level', 'club_position', 'club_jersey_number', 'club_loaned_from', 
                   'club_joined', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 
                   'nation_jersey_number', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 
                   'player_traits', 'player_face_url', 'club_logo_url','club_flag_url', 'nation_logo_url', 'nation_flag_url', 'year', 'gender', 
                   'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 
                   'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'movement_acceleration', 'movement_sprint_speed', 'attacking_finishing', 
                   'power_long_shots', 'attacking_short_passing', 'skill_dribbling', 'skill_ball_control', 'defending_marking_awareness', 
                   'mentality_interceptions', 'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_handling', 'goalkeeping_kicking', 
                   'goalkeeping_positioning', 'goalkeeping_reflexes', 'movement_acceleration', 'movement_sprint_speed', 'preferred_foot', 'potential']

class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns = None):
        super().__init__()
        self.columns = columns
    def _transform(self, dataset):
        output_df = dataset.drop(*self.columns)
        return output_df

class ReplaceNull(Transformer): #this transformer will replace Null cells with integer 0.
    def __init__(self):
        super().__init__()
    def _transform(self, dataset):
        output_df = dataset.na.fill(0)
        return output_df

def process_pipeline():    
    # Removing all unnecessary columbs
    columnDrop = ColumnDropper(columns = columns_to_drop)

    replaceNull = ReplaceNull()

    # Stage where all relevant features are assembled into a vector.
    stage_vector_assembler = VectorAssembler(inputCols=continuous_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

    columnDrop2 = ColumnDropper(columns =  continuous_cols)

    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[columnDrop, replaceNull, stage_vector_assembler, stage_scaler, columnDrop2])

    return pipeline 

In [15]:
#SPLIT THE DATA FOR TESTING, TRAINING, AND VALIDATION
df_test, df_train, df_validate = df_male.randomSplit([0.2, 0.7, 0.1], seed=1234) 

In [16]:
#DEFINING FUNCTION TO RUN THE DATASETS THROUGH THE PIPELINE FOR THE DATA ENGINEERING
def run_pipeline(dataframe):
    preprocess_pipeline = process_pipeline()
    preprocessed = preprocess_pipeline.fit(df_train)
    transformed = preprocessed.transform(dataframe)
    return transformed

train_df = run_pipeline(df_train)
validate_df = run_pipeline(df_validate)
test_df = run_pipeline(df_test)

In [17]:
#SHOWING AN EXAMPLE OF THE DATASET AFTER THE PIPELINE AND DATA ENGINEERING
train_df.show(20)

+-------+--------------------+--------------------+
|overall| vectorized_features|            features|
+-------+--------------------+--------------------+
|     93|[27.0,169.0,67.0,...|[5.80117694718602...|
|     92|[29.0,185.0,80.0,...|[6.23089375808869...|
|     90|[30.0,180.0,80.0,...|[6.44575216354002...|
|     90|[32.0,195.0,95.0,...|[6.87546897444269...|
|     90|[28.0,193.0,92.0,...|[6.01603535263735...|
|     89|[30.0,170.0,65.0,...|[6.44575216354002...|
|     89|[27.0,181.0,81.0,...|[5.80117694718602...|
|     88|[30.0,187.0,71.0,...|[6.44575216354002...|
|     88|[29.0,183.0,79.0,...|[6.23089375808869...|
|     88|[31.0,170.0,72.0,...|[6.66061056899135...|
|     88|[28.0,177.0,72.0,...|[6.01603535263735...|
|     88|[23.0,173.0,74.0,...|[4.94174332538068...|
|     87|[30.0,170.0,66.0,...|[6.44575216354002...|
|     87|[28.0,183.0,75.0,...|[6.01603535263735...|
|     87|[28.0,170.0,67.0,...|[6.01603535263735...|
|     87|[28.0,174.0,65.0,...|[6.01603535263735...|
|     87|[25

In [18]:
#SHOWING AN EXAMPLE OF THE DATASET AFTER THE PIPELINE AND DATA ENGINEERING
test_df.show(20)

+-------+--------------------+--------------------+
|overall| vectorized_features|            features|
+-------+--------------------+--------------------+
|     87|[29.0,183.0,79.0,...|[6.23089375808869...|
|     87|[24.0,183.0,74.0,...|[5.15660173083201...|
|     86|[22.0,199.0,88.0,...|[4.72688491992935...|
|     85|[27.0,188.0,78.0,...|[5.80117694718602...|
|     85|[26.0,174.0,63.0,...|[5.58631854173468...|
|     85|[27.0,184.0,71.0,...|[5.80117694718602...|
|     84|[35.0,177.0,68.0,...|[7.52004419079669...|
|     84|[30.0,170.0,67.0,...|[6.44575216354002...|
|     84|[27.0,192.0,85.0,...|[5.80117694718602...|
|     84|[27.0,188.0,88.0,...|[5.80117694718602...|
|     84|[27.0,189.0,85.0,...|[5.80117694718602...|
|     84|[25.0,171.0,62.0,...|[5.37146013628335...|
|     83|[32.0,196.0,87.0,...|[6.87546897444269...|
|     83|[29.0,186.0,76.0,...|[6.23089375808869...|
|     83|[27.0,181.0,72.0,...|[5.80117694718602...|
|     83|[31.0,185.0,84.0,...|[6.66061056899135...|
|     83|[23

In [19]:
#SHOWING AN EXAMPLE OF THE DATASET AFTER THE PIPELINE AND DATA ENGINEERING
validate_df.show(20)

+-------+--------------------+--------------------+
|overall| vectorized_features|            features|
+-------+--------------------+--------------------+
|     86|[25.0,183.0,76.0,...|[5.37146013628335...|
|     86|[26.0,180.0,70.0,...|[5.58631854173468...|
|     84|[26.0,184.0,82.0,...|[5.58631854173468...|
|     84|[22.0,179.0,67.0,...|[4.72688491992935...|
|     83|[30.0,174.0,74.0,...|[6.44575216354002...|
|     83|[28.0,186.0,78.0,...|[6.01603535263735...|
|     83|[27.0,175.0,70.0,...|[5.80117694718602...|
|     83|[24.0,180.0,74.0,...|[5.15660173083201...|
|     83|[28.0,193.0,84.0,...|[6.01603535263735...|
|     83|[28.0,187.0,84.0,...|[6.01603535263735...|
|     82|[34.0,183.0,83.0,...|[7.30518578534536...|
|     82|[27.0,196.0,91.0,...|[5.80117694718602...|
|     82|[28.0,175.0,71.0,...|[6.01603535263735...|
|     82|[27.0,189.0,86.0,...|[5.80117694718602...|
|     81|[33.0,179.0,75.0,...|[7.09032737989402...|
|     81|[33.0,189.0,75.0,...|[7.09032737989402...|
|     81|[29

In [20]:
#defining function to turn Spark df to Torch
def spark_to_torch(dataframe, x, y):
    pandas_df = dataframe.toPandas()
    x_train = torch.from_numpy(np.array(pandas_df[f'{x}'].values.tolist(), np.float32))
    y_train = torch.from_numpy(np.array(pandas_df[f'{y}'].values.tolist(), np.float32))    
    return x_train, y_train

#converting Spark df to Torch
x_train, y_train = spark_to_torch(train_df, "features", "overall")
x_test, y_test = spark_to_torch (test_df, "features", "overall")
x_validate, y_validate = spark_to_torch(validate_df, "features", "overall")

In [21]:
#CREATING CUSTOM DATALOAD AND DATASET
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset): 
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return (self.x[idx],self.y[idx])

training_set = MyDataset(x_train, y_train)
testing_set = MyDataset(x_test, y_test)
validation_set = MyDataset(x_validate, y_validate)

In [22]:
###
# USING MPS ACCELARATION FOR FASTER PROCESSING
###
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")

# Using the MPS acceleration
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

MPS backend is available!


In [23]:
#SHOWING EXAMPLE SHAPE OF THE DATASET WE HAVE
x_train.shape

torch.Size([99074, 31])

In [24]:
y_train.shape

torch.Size([99074])

In [25]:
x_test.shape

torch.Size([28725, 31])

In [26]:
y_test.shape

torch.Size([28725])

In [27]:
x_validate.shape

torch.Size([14280, 31])

In [28]:
y_validate.shape

torch.Size([14280])

# PYTORCH

## PYTORCH: ML MODEL 1 - MLP

In [29]:
#DEFINING THE CLASS FOR THE MLP MODEL
from torch import nn
class myMultiLayerPerceptron(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.sequential = nn.Sequential(  # here we stack multiple layers together
            nn.Linear(input_dim,28),
            nn.ReLU(),
            nn.Linear(28,20),
            nn.ReLU(),
            nn.Linear(20,15),
            nn.ReLU(),
            nn.Linear(15,10),
            nn.ReLU(),
            nn.Linear(10,5),
            nn.ReLU(),
            nn.Linear(5,output_dim)   
        )
    def forward(self,x):
        y = self.sequential(x)
        return y

In [30]:
mymodel_hyper = myMultiLayerPerceptron(31, 1) # creating a model instance with input dimension 31 and output dimension 1
mymodel_hyper.to(device)
print(mymodel_hyper)

myMultiLayerPerceptron(
  (sequential): Sequential(
    (0): Linear(in_features=31, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=15, bias=True)
    (5): ReLU()
    (6): Linear(in_features=15, out_features=10, bias=True)
    (7): ReLU()
    (8): Linear(in_features=10, out_features=5, bias=True)
    (9): ReLU()
    (10): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [31]:
# building hyperparameters grid to search
lr_values = [0.0005, 0.001, 0.01] 
batch_sizes = [128, 256, 512]
N_epochs = 20

#INITIALIZING THE LOSS FOR LATER COMPARISON
best_validate_loss = float('inf')
best_params = {}

loss_fun = nn.MSELoss() #Mean Square Loss

for lr, batch_size in itertools.product(lr_values, batch_sizes):
    #https://docs.python.org/3/library/itertools.html
    optimizer = torch.optim.Adam(mymodel_hyper.parameters(), lr=lr)

    train_dataloader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    validate_dataloader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

    losses_per_epoch = []
    validate_losses_per_epoch = []
    raw_batch_loss = []
    validate_R2_per_epoch = []

    for epoch in range(N_epochs):
        batch_loss = []
        ###
        # TRAINING LOOP
        ###
        for x_batch, y_batch in train_dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            prediction_score = mymodel_hyper(x_batch).squeeze(-1) 
            loss = loss_fun(prediction_score, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.detach().cpu().numpy())

        ###
        # VALIDATION LOOP
        ###
        targets = []
        predictions = []
        validate_batch_loss = []
        with torch.no_grad():
            for x_batch, y_batch in validate_dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                prediction_score = mymodel_hyper(x_batch).squeeze(-1)
                loss = loss_fun(prediction_score, y_batch)
                validate_batch_loss.append(loss.detach().cpu())
                targets.append(y_batch.cpu())
                predictions.append(prediction_score.cpu())

        ###
        # APPENDING LOSSES and R2
        ###
        losses_per_epoch.append(np.mean(np.array(batch_loss)))
        raw_batch_loss.append(np.array(batch_loss))
        validate_losses_per_epoch.append(np.mean(np.array(validate_batch_loss)))

        targets_per_batch = torch.cat(targets).cpu().numpy()
        predictions_per_batch = torch.cat(predictions).cpu().numpy()

        ssr = np.sum((targets_per_batch - predictions_per_batch)**2)
        sst = np.sum((targets_per_batch - np.mean(targets_per_batch))**2)
        r2 = 1 - ssr/sst
        #https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html

        validate_R2_per_epoch.append(r2)

        print(f"For lr={lr} and batch_size={batch_size}\n, Epoch = {epoch}, train_loss={losses_per_epoch[-1]}, validate_loss = {validate_losses_per_epoch[-1]}, validation R2 = {r2}")

        #CHECKING WHETHER THE CURRENT LOSS IS THE BEST MEANING IS IT THE SMALLEST WE COULD FIND.
        if validate_losses_per_epoch[-1] < best_validate_loss:
            best_validate_loss = validate_losses_per_epoch[-1] #UPDATING THE BEST LOSS
            best_params = {'lr': lr, 'batch_size': batch_size} #SAVING THE BEST MODEL TO DICTIONARY LATER FOR PRINTING
            print(f"Best model so far with lr={lr} and batch_size={batch_size}. Saving Model...\n")
            torch.save(mymodel_hyper.state_dict(), "best_model") #SAVING THE BEST MODEL 


print(f"best hyperparameters: {best_params}")


For lr=0.0005 and batch_size=128
, Epoch = 0, train_loss=642.8499755859375, validate_loss = 20.227306365966797, validation R2 = 0.5909741222858429
Best model so far with lr=0.0005 and batch_size=128. Saving Model...

For lr=0.0005 and batch_size=128
, Epoch = 1, train_loss=17.868736267089844, validate_loss = 15.16547966003418, validation R2 = 0.6934332847595215
Best model so far with lr=0.0005 and batch_size=128. Saving Model...

For lr=0.0005 and batch_size=128
, Epoch = 2, train_loss=13.170848846435547, validate_loss = 11.666879653930664, validation R2 = 0.763959527015686
Best model so far with lr=0.0005 and batch_size=128. Saving Model...

For lr=0.0005 and batch_size=128
, Epoch = 3, train_loss=10.84380054473877, validate_loss = 10.538297653198242, validation R2 = 0.7869528383016586
Best model so far with lr=0.0005 and batch_size=128. Saving Model...

For lr=0.0005 and batch_size=128
, Epoch = 4, train_loss=9.746026992797852, validate_loss = 9.722594261169434, validation R2 = 0.803

In [32]:
best_model = myMultiLayerPerceptron(31, 1)
best_model.to(device)
best_model.load_state_dict(torch.load("best_model"))

/var/folders/c5/p6q0gjzj2ns5lv27n9yp07ph0000gn/T/ipykernel_24555/3870114290.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("best_m

<All keys matched successfully>

In [33]:
#RUNNING TESTING SET ON THE BEST MODEL SO FAR
batch_size = 300
test_dataloader = DataLoader(testing_set,batch_size = batch_size,shuffle = True)
loss_fun = nn.MSELoss()
targets = []
predictions = []
batch_loss = []
with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        prediction_score = best_model(x_batch).squeeze(-1)
        loss = loss_fun(prediction_score, y_batch)
        batch_loss.append(loss.detach().cpu())
        
        targets.append(y_batch.cpu())
        predictions.append(prediction_score.cpu())
            
###
# APPENDNING LOSSES and R2
###
test_loss = np.mean(np.array(batch_loss))

#CALCULATING THE R2
targets_per_batch = torch.cat(targets).numpy()
predictions_per_batch = torch.cat(predictions).numpy()
ssr = np.sum((targets_per_batch - predictions_per_batch)**2)
sst = np.sum((targets_per_batch - np.mean(targets_per_batch))**2)
r2 = 1 - ssr/sst
#https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html
    
print(f"test_loss={test_loss}, R2 = {r2}")

test_loss=1.5188747644424438, R2 = 0.969415195286274


## PYTORCH: ML MODEL 2 - Linear Regression

In [34]:
torch.mps.empty_cache()

In [100]:
#DEFNING THE LINEAR REGRESSION MODEL AND CLASS
from torch import nn

class MyLinearRegressionModel(nn.Module): 
    def __init__(self,d):
        super(MyLinearRegressionModel,self).__init__()
        self.w = nn.Parameter(torch.randn(1,d, dtype=torch.float)) #INITIALIZING TO RANDOM VALUE
        self.b = nn.Parameter(torch.randn(1,dtype=torch.float)) #INITIALIZING TO RANDOM VALUE
    def forward(self,x):
        return torch.inner(x,self.w) + self.b # w^T*x+b

mymodel = MyLinearRegressionModel(31)
print(mymodel.w)
print(mymodel.b)
mymodel.to(device)

Parameter containing:
tensor([[-1.1698,  1.3962,  0.8090,  0.0180,  1.6372, -0.1073, -0.3014, -0.7486,
          1.1378,  1.0596,  1.6638, -2.3976,  2.3913, -0.5689, -0.5526, -0.6279,
         -0.7609,  0.0509, -0.0744,  0.5123,  0.0604,  1.7820,  0.9298,  1.8276,
         -0.3984,  0.1904,  1.3870,  0.0215,  0.7134,  0.6491, -0.2718]],
       requires_grad=True)
Parameter containing:
tensor([-0.9409], requires_grad=True)


MyLinearRegressionModel()

In [101]:
# building hyperparameters grid to search
lr_values = [0.005, 0.01]
batch_sizes = [107, 128]
N_epochs = 20

best_validate_loss = float('inf')
best_params = {}

loss_fun = nn.MSELoss() #loss function

for lr, batch_size in itertools.product(lr_values, batch_sizes):
    #https://docs.python.org/3/library/itertools.html
    optimizer = torch.optim.Adam(mymodel.parameters(), lr=lr)

    train_dataloader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    validate_dataloader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

    losses_per_epoch = []
    validate_losses_per_epoch = []
    raw_batch_loss = []
    validate_R2_per_epoch = []

    for epoch in range(N_epochs):
        batch_loss = []
        ###
        # TRAINING LOOP
        ###
        for x_batch, y_batch in train_dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            prediction_score = mymodel(x_batch).squeeze(-1)
            loss = loss_fun(prediction_score, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.detach().cpu().numpy())

        ###
        # VALIDATION LOOP
        ###
        targets = []
        predictions = []
        validate_batch_loss = []
        with torch.no_grad():
            for x_batch, y_batch in validate_dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                prediction_score = mymodel(x_batch).squeeze(-1)
                loss = loss_fun(prediction_score, y_batch)
                validate_batch_loss.append(loss.detach().cpu())
                targets.append(y_batch.cpu())
                predictions.append(prediction_score.cpu())

        ###
        # APPENDING LOSSES and R2
        ###
        losses_per_epoch.append(np.mean(np.array(batch_loss)))
        raw_batch_loss.append(np.array(batch_loss))
        validate_losses_per_epoch.append(np.mean(np.array(validate_batch_loss)))

        targets_per_batch = torch.cat(targets).numpy()
        predictions_per_batch = torch.cat(predictions).numpy()

        ssr = np.sum((targets_per_batch - predictions_per_batch)**2)
        sst = np.sum((targets_per_batch - np.mean(targets_per_batch))**2)
        r2 = 1 - ssr/sst
        validate_R2_per_epoch.append(r2)
        #https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html

        print(f"For lr={lr} and batch_size={batch_size}\n, Epoch = {epoch}, train_loss={losses_per_epoch[-1]}, validate_loss = {validate_losses_per_epoch[-1]}, R2 = {r2}")


        if validate_losses_per_epoch[-1] < best_validate_loss:
            best_validate_loss = validate_losses_per_epoch[-1]
            best_params = {'lr': lr, 'batch_size': batch_size}
            print(f"Best model so far with lr={lr} and batch_size={batch_size}. Saving Model...\n")
            torch.save(mymodel.state_dict(), "best_model")


print(f"best hyperparameters: {best_params}")


For lr=0.005 and batch_size=107
, Epoch = 0, train_loss=33.327945709228516, validate_loss = 15.042938232421875, R2 = 0.6954439282417297
Best model so far with lr=0.005 and batch_size=107. Saving Model...

For lr=0.005 and batch_size=107
, Epoch = 1, train_loss=12.300430297851562, validate_loss = 10.824694633483887, R2 = 0.7814933657646179
Best model so far with lr=0.005 and batch_size=107. Saving Model...

For lr=0.005 and batch_size=107
, Epoch = 2, train_loss=9.949703216552734, validate_loss = 9.513049125671387, R2 = 0.8073865324258804
Best model so far with lr=0.005 and batch_size=107. Saving Model...

For lr=0.005 and batch_size=107
, Epoch = 3, train_loss=9.164018630981445, validate_loss = 9.061199188232422, R2 = 0.8165120333433151
Best model so far with lr=0.005 and batch_size=107. Saving Model...

For lr=0.005 and batch_size=107
, Epoch = 4, train_loss=8.868366241455078, validate_loss = 8.829041481018066, R2 = 0.8212859779596329
Best model so far with lr=0.005 and batch_size=107

In [102]:
best_model = MyLinearRegressionModel(31)
best_model.to(device)
best_model.load_state_dict(torch.load("best_model"))

/var/folders/c5/p6q0gjzj2ns5lv27n9yp07ph0000gn/T/ipykernel_23963/1133587131.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("best_m

<All keys matched successfully>

In [103]:
batch_size = 300
test_dataloader = DataLoader(testing_set,batch_size = batch_size,shuffle = True)
loss_fun = nn.MSELoss()
targets = []
predictions = []
batch_loss = []
with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        prediction_score = best_model(x_batch).squeeze(-1)
        loss = loss_fun(prediction_score, y_batch)
        batch_loss.append(loss.detach().cpu())
        
        targets.append(y_batch.cpu())
        predictions.append(prediction_score.cpu())
            
###
# APPENDNING LOSSES and R2
###
test_loss = np.mean(np.array(batch_loss))

targets_per_batch = torch.cat(targets).numpy()
predictions_per_batch = torch.cat(predictions).numpy()

ssr = np.sum((targets_per_batch - predictions_per_batch)**2)
sst = np.sum((targets_per_batch - np.mean(targets_per_batch))**2)
r2 = 1 - ssr/sst
#https://www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/coefficient-of-determination-r-squared.html
    
print(f"test_loss={test_loss}, R2 = {r2}")

test_loss=8.47585391998291, R2 = 0.8293067514896393


# SPARK ML MODELLING

## SPARK ML MODEL 1: LINEAR REGRESSION

In [502]:
df_test, df_train = df_male.randomSplit([0.2, 0.8], seed=12)

In [503]:
def run_pipeline(dataframe):
    preprocess_pipeline = process_pipeline()
    preprocessed = preprocess_pipeline.fit(df_train)
    transformed = preprocessed.transform(dataframe)
    return transformed

train_df = run_pipeline(df_train)
test_df = run_pipeline(df_test)

In [504]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

In [505]:
def lin_regression_model(train_df, test_df):
    lr = LinearRegression(featuresCol = 'features', labelCol= 'overall') #linear regression model


    paramGrid = ParamGridBuilder() #create hyperparameter grid
    #Defining parameters for tuning
    param = paramGrid.addGrid(lr.regParam, [0.01, 0.0001])\
                 .addGrid(lr.maxIter, [20, 40])\
                 .build()

    evaluator = RegressionEvaluator(labelCol = "overall", predictionCol = "prediction", metricName = "rmse") #evaluating based on RMSE

    crossval = CrossValidator(estimator=lr, #performing cross validation
                              estimatorParamMaps=param,
                              evaluator = evaluator,
                              numFolds=3)

    cv_model = crossval.fit(train_df) # Fit the model and pick the best performing one
    best_model = cv_model.bestModel

    lr_preds = best_model.transform(test_df)

    lr_rmse = evaluator.evaluate(lr_preds)

    print(f"Best Linear Regression Model: {best_model} and for this model, Regression RMSE: {lr_rmse}")

    return best_model

In [506]:
lin_regression_model(train_df, test_df)

Best Linear Regression Model: LinearRegressionModel: uid=LinearRegression_589a479cb1aa, numFeatures=31 and for this model, Regression RMSE: 2.8978227408594064


LinearRegressionModel: uid=LinearRegression_589a479cb1aa, numFeatures=31

## SPARK ML MODEL 2: RANDOM FOREST MODEL

In [507]:
def random_forest_model(train_df, test_df):

    rfr = RandomForestRegressor(featuresCol = 'features', labelCol= 'overall') #Random Forest Regression model is created

    paramGrid = ParamGridBuilder() #create hyperparameter grid
    #Defining RFR Model parameters for tuning
    param = paramGrid.addGrid(rfr.numTrees, [10, 20])\
                 .addGrid(rfr.maxDepth, [5, 10])\
                 .addGrid(rfr.maxBins, [32, 64])\
                 .build()

    evaluator = RegressionEvaluator(labelCol = "overall", predictionCol = "prediction", metricName = "rmse") #evaluate the model on rmse

    crossval = CrossValidator(estimator=rfr, #performing cross validation
                              estimatorParamMaps=param,
                              evaluator = evaluator,
                              numFolds=3)
   
    cv_model = crossval.fit(train_df) # Fit the model and pick the best performing one
    best_model = cv_model.bestModel

    rfr_preds = best_model.transform(test_df)

    rfr_rmse = evaluator.evaluate(rfr_preds)

    print(f"Best Random Forest Regression Model: {best_model} and for this model, Regression RMSE: {rfr_rmse}")

    return best_model

In [508]:
random_forest_model(train_df, test_df)

Best Random Forest Regression Model: RandomForestRegressionModel: uid=RandomForestRegressor_b20863e19b26, numTrees=20, numFeatures=31 and for this model, Regression RMSE: 1.6701996590600843


RandomForestRegressionModel: uid=RandomForestRegressor_b20863e19b26, numTrees=20, numFeatures=31